In [1]:
import gym
import numpy as np
import torch
import ptan
import common
import atari_wrappers
from torch.utils.tensorboard import SummaryWriter
import ptan_copy



EPSILON = 1.0
GAMMA = 0.99
REPLAY_SIZE = 2000
TGT_NET_SYNC = 10
EPS_DECAY = 0.96
BATCH_SIZE = 800

env = gym.make("Breakout-v4")
env = atari_wrappers.reshapeWrapper(env)
env = atari_wrappers.ScaledFloatFrame(env)
env = atari_wrappers.oldWrapper(env)
env = atari_wrappers.MaxAndSkipEnv(env)
env = atari_wrappers.DumbRewardWrapper(env)


PATH = "Breakout-v4.pt"
device = "cpu"

net = common.DQN((3, 210,160), env.action_space.n).to(device)
tgt_net = ptan.agent.TargetNet(net)

preprocessor = common.ndarray_preprocessor(common.VariableTensor_preprocessor())
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=EPSILON)
agent = ptan.agent.DQNAgent(net, action_selector=selector, device=device, preprocessor=preprocessor)
exp_source = ptan.experience.ExperienceSourceFirstLast(env, agent, gamma=0.0)
buffer = ptan.experience.ExperienceReplayBuffer(exp_source, buffer_size=REPLAY_SIZE)
optimizer = torch.optim.Adam(net.parameters())

c:\Users\ianmi\miniconda3\envs\GPUtorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
exp_source2 = ptan_copy.experience.ExperienceSourceFirstLast(env, agent,gamma=0.5)

idx = 0
done_id = 0
episode = 0
def unpack_batch(batch): # return states, actions, calculated tgt_q_v = r + tgt_net(last_state)*GAMMA
    states = []
    rewards = []
    actions = []
    last_states = []
    for exp in batch:
        states.append(exp.state)
        rewards.append(exp.reward)
        actions.append(exp.action)
        last_states.append(exp.last_state)
    print(len(states), rewards)
    

while True:
    buffer.populate(3)
    batch = buffer.sample(3)
    states_v, actions_v, tgt_states_v = unpack_batch(batch) # *elements are state, action, reward, last_state 
    


    #(verify that the reward is truly immediate by verifying from the function __iter__ that states1 == returned_states but returned r from r1 and r2 discounted is = r1)
    #also look for GAMMA effect (default should be 1 for desired no discounting)
    break


    '''
    for rewards, steps in exp_source.pop_rewards_steps():
        episode +=1
        print("idx %d, steps %d, episode %d done, reward=%.3f rewards, epsilon=%.2f" %(idx,steps,episode, rewards, selector.epsilon))
        solved = rewards > 150
    if solved:
        print("Done in %d idx" % idx)
        break
    if len(buffer) < 2*BATCH_SIZE:
        continue

    batch = buffer.sample(BATCH_SIZE)
    states_v, actions_v, tgt_q_v = unpack_batch(
    batch, tgt_net.target_model, GAMMA)
    optimizer.zero_grad()
    q_v = net(states_v)
    q_v = q_v.gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
    loss_v = torch.nn.functional.mse_loss(q_v, tgt_q_v)
    loss_v.backward()
    optimizer.step()
    selector.epsilon *= EPS_DECAY

    if step % TGT_NET_SYNC == 0:
        tgt_net.sync()'''

3 [1.0, 1.0, 1.0]


TypeError: cannot unpack non-iterable NoneType object